In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date, cast

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/24 21:38:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/24 21:38:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_fhvhv = spark.read.parquet('fhvhv/2021/06/*')

In [3]:
df_fhvhv = df_fhvhv.withColumn('date_only', to_date(col('pickup_datetime')))

In [4]:
df_fhvhv = df_fhvhv \
    .withColumn('DiffInSeconds',col("dropoff_datetime").cast("long") - col('pickup_datetime').cast("long")) \
    .withColumn('DiffInHours',(col('DiffInSeconds')/3600))

In [5]:
df_fhvhv.registerTempTable('trips_data')

/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
spark.sql("""
SELECT
    date_only,
    count(1)
FROM
    trips_data
GROUP BY 
    date_only
ORDER BY 
    date_only
""").show()

+----------+--------+
| date_only|count(1)|
+----------+--------+
|2021-06-01|  417375|
|2021-06-02|  457339|
|2021-06-03|  521408|
|2021-06-04|  538917|
|2021-06-05|  604903|
|2021-06-06|  522753|
|2021-06-07|  425771|
|2021-06-08|  462554|
|2021-06-09|  483353|
|2021-06-10|  504108|
|2021-06-11|  549286|
|2021-06-12|  591339|
|2021-06-13|  509039|
|2021-06-14|  426672|
|2021-06-15|  452470|
|2021-06-16|  479776|
|2021-06-17|  497133|
|2021-06-18|  540056|
|2021-06-19|  601189|
|2021-06-20|  491630|
+----------+--------+
only showing top 20 rows



In [7]:
spark.sql("""
SELECT
    date_only,
    DiffInSeconds,
    DiffInHours
FROM 
    trips_data
ORDER BY
    DiffInSeconds DESC

""").show()

+----------+-------------+------------------+
| date_only|DiffInSeconds|       DiffInHours|
+----------+-------------+------------------+
|2021-06-25|       240764|  66.8788888888889|
|2021-06-22|        91979|25.549722222222222|
|2021-06-27|        71931|19.980833333333333|
|2021-06-26|        65510|18.197222222222223|
|2021-06-23|        59281|16.466944444444444|
|2021-06-23|        51368|14.268888888888888|
|2021-06-24|        50075|13.909722222222221|
|2021-06-04|        42012|             11.67|
|2021-06-27|        40917|11.365833333333333|
|2021-06-20|        39544|10.984444444444444|
|2021-06-01|        36963|           10.2675|
|2021-06-28|        35879| 9.966388888888888|
|2021-06-01|        35879| 9.966388888888888|
|2021-06-27|        34696| 9.637777777777778|
|2021-06-18|        34648| 9.624444444444444|
|2021-06-08|        34129| 9.480277777777777|
|2021-06-11|        34098| 9.471666666666666|
|2021-06-15|        33848| 9.402222222222223|
|2021-06-25|        33817| 9.39361